In [43]:
import os
import openpyxl
import locale
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_OUTROS')

In [44]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        

In [45]:
def query_dados(conn, table):  
    query = (f"""
        (
            SELECT
                SUM(mprd.mprd_valor) AS valor_liq,
                mprc.mprc_fpgt_codigo AS cod_fpgt,
                fpgt.fpgt_descricao AS fpgt_nome
            FROM {table} AS mprd 
            INNER JOIN movprodc AS mprc ON mprd.mprd_transacao = mprc.mprc_transacao
            INNER JOIN fpgto AS fpgt ON fpgt.fpgt_codigo = mprc.mprc_fpgt_codigo
            WHERE mprd_status = 'N' 
            AND mprd.mprd_dcto_codigo IN ('6666','6667','6668','7339','7335','7338','7337')
            GROUP BY mprc.mprc_fpgt_codigo, fpgt.fpgt_descricao
        )  
    """)
    
    df = pd.read_sql_query(query, conn)
       
    return df

In [46]:
def gerar_excel(df, ws):
    df_resumo = df.groupby(['cod_fpgt','fpgt_nome'], as_index=False)['valor_liq'].sum()

    ws.title = f"F.Pgto"
  
    columns_title = [
        'CodPgto','Descrição','Valor'
    ]
    
    for col_num, column_title in enumerate(columns_title, 1):
        ws.cell(row=1, column=col_num).value = column_title

    for index, row in enumerate(df_resumo.itertuples(), start=2):
        ws.cell(row=index, column=1).value = row.cod_fpgt
        ws.cell(row=index, column=2).value = row.fpgt_nome
        ws.cell(row=index, column=3).value = row.valor_liq
        ws.cell(row=index, column=3).number_format = 'R$ #,##0.00'


In [47]:
def teste_loop():
    conn = get_db_engine()
       
    locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')

    tables = [
        'movprodd0123', 'movprodd0223', 'movprodd0323', 'movprodd0423', 'movprodd0523', 'movprodd0623', 
        'movprodd0723', 'movprodd0823', 'movprodd0923', 'movprodd1023', 'movprodd1123', 'movprodd1223'
    ]
    wb = openpyxl.Workbook()
    
    for table in tables:
        df = query_dados(conn, table)

        ws = wb.create_sheet(title=table)
        ws.append(['CodPgto', 'NomePgto', 'ValorTotal'])

        for row in df.itertuples(index=False):
            ws.append([row.cod_fpgt, row.fpgt_nome, row.valor_liq])
            ws.cell(row=ws.max_row, column=3).number_format = 'R$ #,##0.00'
    
    if 'Sheet' in wb.sheetnames:
        wb.remove(wb['Sheet'])
    
    data_pasta = datetime.now().strftime("%Y-%m-%d")
    nome_arquivo = f'F.Pgto-{data_pasta}.xlsx'
    diretorio = os.path.join(path_dados, 'outros', data_pasta)
    if not os.path.exists(diretorio):
        os.mkdir(diretorio)
    local_arquivo = os.path.join(diretorio, nome_arquivo)
    wb.save(local_arquivo)

teste_loop()

Conexão estabelecida!
Banco de dados conectado!
